In [15]:
import numpy as np
import pandas as pd
from jupyter_dash import JupyterDash
import plotly.express as px
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash import Input, Output
from dash.exceptions import PreventUpdate
import plotly


ImportError: cannot import name 'escape' from 'jinja2' (C:\Users\koray\anaconda3\lib\site-packages\jinja2\__init__.py)

In [ ]:
emission = pd.read_csv('EmissionData.csv', index_col=0)

In [ ]:
def classify(df):
    df["Emission Class"] = np.empty_like(df["Emission"], dtype=object)
    for i in range(len(df["Emission"])):
        if np.isnan(df.loc[i, "Emission"]) or df.loc[i, "Emission"] < 2.21:
            df.loc[i, "Emission Class"] = '<2.21'
        elif df.loc[i, "Emission"] < 5.10:
            df.loc[i, "Emission Class"] = '2.21-5.10'
        elif df.loc[i, "Emission"] < 9.64:
            df.loc[i, "Emission Class"] = '5.10-9.64'
        elif df.loc[i, "Emission"] < 16.64:
            df.loc[i, "Emission Class"] = '9.64-16.64'
        elif df.loc[i, "Emission"] >= 16.64:
            df.loc[i, "Emission Class"] = '>16.64'
    return df

emission = classify(df=emission)

In [1]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.Br(),
    html.H1('Carbon Index - World Bank Data', style={'textAlign': 'center'}),
    html.Br(),
    dbc.Row([
        dbc.Col([
            dbc.Label("By Country"),
            dcc.Dropdown(
                id='emission_country_dropdown',
                multi=True,
                options=[{'label': country, 'value': country} for country in emission['Country'].drop_duplicates().sort_values()]
            ),
            dcc.Graph(id='emission_line'),
            html.Br(),
            dbc.Label('Select Year Range'),
            dcc.RangeSlider(
                id='emission_year_slider',
                min=emission["Year"].min(),
                max=emission["Year"].max(),
                step=1,
                marks=None,
                tooltip={"placement": "bottom", "always_visible": True},
                value=[emission["Year"].min(), emission["Year"].max()]
            ),
        ], width=6),
        dbc.Col([
            dbc.Label("By Year"),
            dcc.Dropdown(
                id='emission_year_dropdown',
                multi=False,
                options=[{'label': str(year), 'value': year} for year in emission['Year'].unique()]
            ),
            dcc.Tabs([
                dcc.Tab(
                    label="Bar",
                    children=[
                        dcc.Loading(
                            type="cube",
                            children=[dcc.Graph(id="emission_bar")]
                        )
                    ]
                ),
                dcc.Tab(
                    label="Map",
                    children=[
                        dcc.Loading(
                            type="cube",
                            children=[dcc.Graph(id="emission_map")]
                        )
                    ]
                ),
            ]),
        ], width=6),
    ]),
])


@app.callback(Output('emission_line', 'figure'),
              [Input('emission_country_dropdown', 'value'),
               Input('emission_year_slider', 'value')])
def update_emission_line(country, year):
    if not country:
        raise PreventUpdate
        
    filtered_country = emission[emission['Country'].isin(country)].dropna(subset=["Country"])
    df_line = filtered_country[(filtered_country['Year']>=year[0]) & (filtered_country['Year']<=year[1])].dropna(subset=["Year"])

    fig = px.line(
        df_line,
        x="Year",
        y='Emission',
        color='Country',
        title=' - '.join(country) + ' - Carbon Emission'
    )

    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    return fig


@app.callback(Output('emission_bar', 'figure'),
              [Input('emission_country_dropdown', 'value'),
              Input('emission_year_dropdown', 'value')])
def update_emission_bar(country,year):
    if not year:
        raise PreventUpdate
    
    filtered_country = emission[emission['Country'].isin(country)].dropna(subset=["Country"])
    df_bar = filtered_country[filtered_country['Year'].eq(year)].dropna(subset=["Year"])

    fig = px.bar(
        df_bar,
        x="Country",
        y='Emission',
        color='Country',
         title=' - '.join(country) +' Carbon Emission' +  ' - Year ' + str(year)
    )

    return fig


@app.callback(Output('emission_map', 'figure'),
              Input('emission_year_dropdown', 'value'))
def update_emission_map(year):
    if not year:
        raise PreventUpdate

    df_map = emission[emission['Year'].eq(year)].dropna(subset=["Year"]).sort_values(by="Emission")

    fig = px.choropleth(df_map, 
                        locations="Country Code",
                        color="Emission Class", 
                        hover_name="Country",
                        hover_data=["Emission Class","Country Code","Emission"],
                        color_discrete_sequence=['rgb(222,235,247)','rgb(198,219,239)','rgb(107,174,214)', 'rgb(33,113,181)','rgb(8,48,107)'],
                        title="Carbon Emission in Year " + str(2010)
                       
                       )


    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.0,
        xanchor="right",
        x=1.1
    ))

    return fig

if __name__ == '__main__':
    app.run_server(mode="external", port=3030)


Dash is running on http://127.0.0.1:3030/

Dash app running on http://127.0.0.1:3030/
